In [ ]:
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/My Drive/Colab Notebooks/CL/assign5/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# General information

1. The python envirentment is the defaut envirentment in Google Colab. Python version: 3.6.9 .
2. Wall time: 3h 45min 12s




In [ ]:
import numpy as np
from itertools import chain

# Define paths

In [ ]:
# define your own rootpath
rootpath = ''
paths = {
    'movie': rootpath+'movies-pp.txt',
}

#Prepare data

In [ ]:
file = open(paths['movie'])
text = [line[:-1].split(' ') for line in file if len(line) > 10]
# text=text[:3]
vocab = list(set(chain(*text)))

n_docs = len(text)  # the number of documents
n_vocab = len(vocab)  # the number of unique words
word2index = dict(zip(vocab, range(n_vocab)))  # lookup
index2word = dict(zip(range(n_vocab), vocab))
# convert word to its coresponding index
text = [[word2index[text[m][n]]
         for n in range(len(text[m]))]for m in range(n_docs)]

n_docs, n_vocab

(2000, 46517)

# Gibbs sampling

extra: using numpy

In [ ]:
def initialize(n_topics):
    global topic_mn
    global docs_topic
    global topic_word
    topic_mn = []  # topic_mn[m][n]: the topic of the word n in the document m
    # docs_topic[m,k]: count of the topic k in the document m
    docs_topic = np.zeros((n_docs, n_topics))
    # topic_word[k,n]: count of the word n in the topic k
    topic_word = np.zeros((n_topics, n_vocab))
    topic_prob = np.ones(n_topics)/n_topics
    for m in range(n_docs):
        topic_n = []
        for n in range(len(text[m])):
            # randomly assign a topic to a word
            k = np.random.multinomial(1, topic_prob).argmax()
            topic_n.append(k)  # record the topic of the word
            docs_topic[m, k] += 1
            topic_word[k, text[m][n]] += 1
        topic_mn.append(topic_n)


def gibbs(alpha, beta, iteration, n_topics):
    for iter in range(iteration):
        for m in range(n_docs):
            for n in range(len(text[m])):
                k = topic_mn[m][n]
                docs_topic[m, k] -= 1
                topic_word[k, text[m][n]] -= 1

                # sampling according to formula 5
                theta = (docs_topic[m, :]+alpha) / \
                    (docs_topic[m, :]+alpha).sum()  # （1，10）
                phi = (topic_word[:, text[m][n]]+beta) / \
                    (topic_word[:, text[m][n]]+beta).sum()
                p_mn = theta*phi
                p_mn = p_mn/p_mn.sum()

                # update the topic of the word
                k = np.random.multinomial(1, p_mn).argmax()
                topic_mn[m][n] = k

                docs_topic[m, k] += 1
                topic_word[k, text[m][n]] += 1
        if iter % 50 == 0:
            print(f'ititeration:{iter} is done')

# get topk for each topic


def get_topics(num):
    c = np.argsort(topic_word, axis=1)[:, :-num-1:-1]
    for i in range(n_topics):
        words = []
        for j in range(num):
            words.append(index2word[c[i, j]])
        print(f'topic {i} : {words}')

In [ ]:
%% time
# hyper parameters
alpha = 0.02
beta = 0.1
n_topics = 20  # the number of topics
iteration = 500

initialize(n_topics)
gibbs(alpha, beta, iteration, n_topics)
get_topics(10)

ititeration:0 is done
ititeration:50 is done
ititeration:100 is done
ititeration:150 is done
ititeration:200 is done
ititeration:250 is done
ititeration:300 is done
ititeration:350 is done
ititeration:400 is done
ititeration:450 is done
topic 0 : ['reusing', 'like', 'high-society', '83', 'chariot', 'puerto', 'sworn', 'gore', 'denigrates', 'godless']
topic 1 : ['foretold', 'cottwell', 'reusing', 'godless', 'high-society', '83', 'chariot', 'puerto', 'sworn', 'gore']
topic 2 : ['loxley', 'jitters', 'engrossing', 'argento', '83', 'chariot', 'puerto', 'sworn', 'gore', 'denigrates']
topic 3 : ['film', 'movie', 'one', 'like', 'even', 'good', 'time', 'would', 'story', 'much']
topic 4 : ['reusing', 'like', 'high-society', '83', 'chariot', 'puerto', 'sworn', 'gore', 'denigrates', 'godless']
topic 5 : ['agamemnon', 'reusing', 'denigrates', 'like', 'uncleanness', 'high-society', '83', 'chariot', 'puerto', 'sworn']
topic 6 : ['reusing', 'like', 'high-society', '83', 'chariot', 'puerto', 'sworn', 'g